# 1.환경설정

In [5]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import requests #naver CLOVA Speech API
import json     #naver CLOVA Speech API
import mecab_ko as MeCab
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import csv
import pandas as pd
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np

# 2.함수 설정

In [12]:
class ClovaSpeechClient:
    # Clova Speech invoke URL
    invoke_url = 'https://clovaspeech-gw.ncloud.com/external/v1/6542/2a76879d93ca2fd6fc6fcf7cdc1cdf1888f7487dbff7c21c5a0ab0d655659361'
    # Clova Speech secret key
    secret = 'e528cad5fdc14d84808b304e6eb77b35'

    def req_upload(self, file, completion='sync', callback=None, userdata=None, forbiddens=None, boostings=None,
                   wordAlignment=True, fullText=True, diarization=None):
        request_body = {
            'language': 'ko-KR',
            'completion': completion,
            'callback': callback,
            'userdata': userdata,
            'wordAlignment': wordAlignment,
            'fullText': fullText,
            'forbiddens': forbiddens,
            'boostings': boostings,
            'diarization': diarization,
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'X-CLOVASPEECH-API-KEY': self.secret
        }
        print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
        files = {
            'media': open(file, 'rb'),
            'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
        }
        response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
        return response


# 텍스트 파일을 읽어서 반환하는 함수
def read_text_file(file_path):
    """file_path를 열어서 text로 반환"""
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# MeCab을 사용하여 텍스트를 형태소로 분석하고 XLSX 파일로 저장하는 함수
def analyze_text_with_mecab_to_xlsx(text, output_xlsx):
    """MeCab을 사용하여 텍스트를 형태소로 분석하고 XLSX 파일로 저장하는 함수"""
    mecab = MeCab.Tagger()
    pos_lists = {'고유명사': [], '일반명사': [], '대명사': [], '동사': [], '형용사': [], '부사': [], '접속사': [], '조사': [], '숫자': []}
    pos_unique_lists = {'고유명사': set(), '일반명사': set(), '대명사': set(), '동사': set(), '형용사': set(), '부사': set(), '접속사': set(), '조사': set(), '숫자': set()}

    # 형태소 분석
    parsed = mecab.parse(text)
    for line in parsed.split('\n'):
        if '\t' not in line:
            continue
        word, tag_info = line.split('\t')
        tag = tag_info.split(',')[0]

        # 품사별 리스트에 단어 추가
        # 고유명사
        if tag == 'NNP':
            pos_lists['고유명사'].append(word)
            pos_unique_lists['고유명사'].add(word)
        # 일반명사
        elif tag == 'NNG':
            pos_lists['일반명사'].append(word)
            pos_unique_lists['일반명사'].add(word)
        # 대명사
        elif tag == 'NP':
            pos_lists['대명사'].append(word)
            pos_unique_lists['대명사'].add(word)
        # 동사
        elif tag == 'VV':
            pos_lists['동사'].append(word)
            pos_unique_lists['동사'].add(word)
        # 형용사
        elif tag == 'VA':
            pos_lists['형용사'].append(word)
            pos_unique_lists['형용사'].add(word)
        # 부사
        elif tag == 'MAG':
            pos_lists['부사'].append(word)
            pos_unique_lists['부사'].add(word)
        # 접속사
        elif tag == 'JC':
            pos_lists['접속사'].append(word)
            pos_unique_lists['접속사'].add(word)
        # 조사
        elif tag == 'JX':
            pos_lists['조사'].append(word)
            pos_unique_lists['조사'].add(word)
        # 숫자
        elif tag == 'SN':
            pos_lists['숫자'].append(word)
            pos_unique_lists['숫자'].add(word)

    # 결과 데이터를 DataFrame으로 변환
    data = {
        '품사': [],
        '단어 수': [],
        '중복 없는 단어 수': [],
        '단어 리스트': [],
        '중복 없는 단어 리스트': []
    }
    for pos in pos_lists:
        data['품사'].append(pos)
        data['단어 수'].append(len(pos_lists[pos]))
        data['중복 없는 단어 수'].append(len(pos_unique_lists[pos]))
        data['단어 리스트'].append(', '.join(pos_lists[pos]))
        data['중복 없는 단어 리스트'].append(', '.join(pos_unique_lists[pos]))

    df = pd.DataFrame(data)



# Function to extract the specified part
def extract_last_part(path):
    """ m4a의 파일 이름 last_part'"""
    # Split the path by '/'
    parts = path.split('/')
    # Take the last part and split by '.txt'
    last_part = parts[-1].split('.m4a')[0]
    return last_part

# 3.Speech-To-Text (feat. CLOVA)

In [13]:
date_num = 'd10'

In [34]:
from datetime import datetime, timedelta


In [135]:
import os
import pandas as pd
import json
import zipfile
import pymysql
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from plugins.database import con_db

# MySQL 연결 정보
db_user = 'root'
db_password = '1234qwer'
db_host = '127.0.0.1'
db_port = 3306  # MySQL 포트
db_name = 'STT'

# MySQL 연결 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# DataFrame을 MySQL 테이블로 적재
table_name = 'STT'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

def get_user_name(file_path):
    # 파일명에서 마지막 부분을 추출하여 user_id와 user_name을 얻어옴
    file_name = os.path.basename(file_path)
    parts = file_name.split('_')
    user_info = parts[1]
    return user_info


def get_user_id(user_name):
    # MySQL에서 user_id에 해당하는 사용자 정보 조회
    connection = pymysql.connect(
        user='root',
        password='1234qwer',
        host='127.0.0.1',
        database='STT',
        cursorclass=pymysql.cursors.DictCursor
    )
    
    cursor = connection.cursor()
    query = f"SELECT user_id FROM user WHERE user_name = '{user_name}'"
    cursor.execute(query)
    result = cursor.fetchone()
    
    cursor.close()
    connection.close()

    return str(result['user_id'])


# 그 외 코드 부분은 유지
def get_m4a_files_in_folder(folder_path, start_with_beforeday=False):
    """ 전날의 .m4a파일을 불러오는 함수"""
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = yesterday.strftime("%y%m%d")

    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.m4a')]
    
    if start_with_beforeday:
        filtered_files = [file for file in files if os.path.basename(file).startswith(yesterday)]
        return filtered_files
    
    return files

output_zipname = f"{date_num}_Quan.zip"

# 폴더 경로 지정
folder_path = '/Users/choi-junyong/github/connectslab/m4a/' + date_num

# 폴더 내의 모든 m4a 파일 가져오기
m4a_files = get_m4a_files_in_folder(folder_path, start_with_beforeday=True)

# 생성된 파일들을 저장할 리스트
created_files = []


# 각 파일에 대해 처리
for audio_file in m4a_files:
    user_info = extract_last_part(audio_file) 
    base_filename = f"{user_info}_"

    res = ClovaSpeechClient().req_upload(file=audio_file, completion='sync')
    clova_output = res.text
    data = json.loads(clova_output)
    segments = data['segments']

    # datetime 필요 filename의 앞 6글자
    data_list = []
    for segment in segments:
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text']
        confidence = segment['confidence']
        speaker_label = segment['speaker']['label']
        text_edited = segment['textEdited']
        user_name = get_user_name(audio_file)
        user_id = get_user_id(user_name)
        date = base_filename[:6]

        data_list.append([user_id, user_name, start_time, end_time, text, confidence, speaker_label, text_edited, date])


    df = pd.DataFrame(data_list, columns=['user_id', 'user_name', '시작 시간', '종료 시간', '텍스트', '신뢰도', '화자', '텍스트 편집 여부', '날짜'])


b'{"language": "ko-KR", "completion": "sync", "callback": null, "userdata": null, "wordAlignment": true, "fullText": true, "forbiddens": null, "boostings": null, "diarization": null}'


In [136]:
df

,user_id,user_name,시작 시간,종료 시간,텍스트,신뢰도,화자,텍스트 편집 여부,날짜
0,000002,김이슬,1970,12225,엄마랑 오늘은 그림을 한번 그려보자. 선유 그림 그릴 거야.,0.966039,1,엄마랑 오늘은 그림을 한번 그려보자. 선유 그림 그릴 거야.,240114
1,000002,김이슬,12225,21290,벌써 잡았구나. 생년필 생연필 이거 중에서 또 어떤 색깔 쓰고 싶어? 선이 어떤 색깔?,0.918971,1,벌써 잡았구나. 생년필 생연필 이거 중에서 또 어떤 색깔 쓰고 싶어? 선이 어떤 색깔?,240114
2,000002,김이슬,24450,34260,이거 핑크 색깔을 높이 들었네 어? 어 저거 어떤 거,0.797421,1,이거 핑크 색깔을 높이 들었네 어? 어 저거 어떤 거,240114
3,000002,김이슬,35320,45130,어떤 색깔 하고 싶어요? 이거 이 색깔 빨간 색깔이 골랐네 빨간색 이걸로 할까?,0.932586,1,어떤 색깔 하고 싶어요? 이거 이 색깔 빨간 색깔이 골랐네 빨간색 이걸로 할까?,240114
4,000002,김이슬,46450,56950,빨간색 엄마는 그러면 보라색깔 엄마는 보라색깔 유도 선유가 하고 싶어 보라 색깔,0.895416,1,빨간색 엄마는 그러면 보라색깔 엄마는 보라색깔 유도 선유가 하고 싶어 보라 색깔,240114
...,...,...,...,...,...,...,...,...,...
64,000002,김이슬,572255,578030,이제 발로 굴려볼 거야. 발로 생연필 발로 굴려볼 거야.,0.786698,1,이제 발로 굴려볼 거야. 발로 생연필 발로 굴려볼 거야.,240114
65,000002,김이슬,578450,587640,대굴대굴 굴러가라. 생연필아 대굴대굴 대굴대굴대굴대굴 대굴대굴대굴대굴 재밌어.,0.752664,1,대굴대굴 굴러가라. 생연필아 대굴대굴 대굴대굴대굴대굴 대굴대굴대굴대굴 재밌어.,240114
66,000002,김이슬,587640,596190,발로 해보니까 재밌어. 골대골대골골골골. 재밌어.,0.688003,1,발로 해보니까 재밌어. 골대골대골골골골. 재밌어.,240114
67,000002,김이슬,596680,601940,엄마가 발로 해줄게.,0.931607,1,엄마가 발로 해줄게.,240114


In [102]:
def get_user_info(user_name):
    # MySQL에서 user_id에 해당하는 사용자 정보 조회
    connection = pymysql.connect(
        user='root',
        password='1234qwer',
        host='127.0.0.1',
        database='STT',
        cursorclass=pymysql.cursors.DictCursor
    )
    
    cursor = connection.cursor()
    query = f"SELECT user_id FROM user WHERE user_name = '{user_name}'"
    cursor.execute(query)
    result = cursor.fetchone()
    
    cursor.close()
    connection.close()

    return result['user_id']

In [103]:
a= get_user_info(user_name)
a

'000002'

In [29]:
import pymysql

In [30]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL 연결 정보
db_user = 'root'
db_password = '1234qwer'
db_host = '127.0.0.1'
db_port = 3306  # MySQL 포트
db_name = 'STT'

df

# MySQL 연결 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# DataFrame을 MySQL 테이블로 적재
table_name = 'STT'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f'DataFrame이 {table_name} 테이블에 적재되었습니다.')


DataFrame이 STT 테이블에 적재되었습니다.


In [32]:

#user id 추가
#하나의 table로 
#화자 1,2 
df

,시작 시간,종료 시간,텍스트,신뢰도,화자,텍스트 편집 여부,날짜
0,1970,12225,엄마랑 오늘은 그림을 한번 그려보자. 선유 그림 그릴 거야.,0.966039,1,엄마랑 오늘은 그림을 한번 그려보자. 선유 그림 그릴 거야.,231215
1,12225,21290,벌써 잡았구나. 생년필 생연필 이거 중에서 또 어떤 색깔 쓰고 싶어? 선이 어떤 색깔?,0.918971,1,벌써 잡았구나. 생년필 생연필 이거 중에서 또 어떤 색깔 쓰고 싶어? 선이 어떤 색깔?,231215
2,24450,34260,이거 핑크 색깔을 높이 들었네 어? 어 저거 어떤 거,0.797421,1,이거 핑크 색깔을 높이 들었네 어? 어 저거 어떤 거,231215
3,35320,45130,어떤 색깔 하고 싶어요? 이거 이 색깔 빨간 색깔이 골랐네 빨간색 이걸로 할까?,0.932586,1,어떤 색깔 하고 싶어요? 이거 이 색깔 빨간 색깔이 골랐네 빨간색 이걸로 할까?,231215
4,46450,56950,빨간색 엄마는 그러면 보라색깔 엄마는 보라색깔 유도 선유가 하고 싶어 보라 색깔,0.895416,1,빨간색 엄마는 그러면 보라색깔 엄마는 보라색깔 유도 선유가 하고 싶어 보라 색깔,231215
...,...,...,...,...,...,...,...
64,572255,578030,이제 발로 굴려볼 거야. 발로 생연필 발로 굴려볼 거야.,0.786698,1,이제 발로 굴려볼 거야. 발로 생연필 발로 굴려볼 거야.,231215
65,578450,587640,대굴대굴 굴러가라. 생연필아 대굴대굴 대굴대굴대굴대굴 대굴대굴대굴대굴 재밌어.,0.752664,1,대굴대굴 굴러가라. 생연필아 대굴대굴 대굴대굴대굴대굴 대굴대굴대굴대굴 재밌어.,231215
66,587640,596190,발로 해보니까 재밌어. 골대골대골골골골. 재밌어.,0.688003,1,발로 해보니까 재밌어. 골대골대골골골골. 재밌어.,231215
67,596680,601940,엄마가 발로 해줄게.,0.931607,1,엄마가 발로 해줄게.,231215


- ⚠️ 음성녹음 파일 경로지정

In [ ]:
# 음성녹음 파일 경로 지정
audio_file = '/content/drive/Shareddrives/커넥츠랩/서비스 테스트(2312)/Recordings/d01/231211_김이슬_TestDay01.m4a'

# 'm4a' 파일명으로부터 user_info 추출
user_info = extract_last_part(audio_file)
base_filename = f"{user_info}_"

print(base_filename)

In [ ]:
import os
import pandas as pd
import json

# 특정 폴더 내의 모든 m4a 파일을 가져오는 함수
def get_m4a_files_in_folder(folder_path):
    return [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.m4a')]

# 폴더 경로 지정
folder_path = '/content/drive/Shareddrives/커넥츠랩/서비스 테스트(2312)/Recordings/d06'

# 폴더 내의 모든 m4a 파일 가져오기
m4a_files = get_m4a_files_in_folder(folder_path)

# 각 파일에 대해 처리
for audio_file in m4a_files:
    user_info = extract_last_part(audio_file)  # 'extract_last_part' 함수는 정의되어 있어야 함
    base_filename = f"{user_info}_"

    res = ClovaSpeechClient().req_upload(file=audio_file, completion='sync')
    clova_output = res.text
    data = json.loads(clova_output)
    segments = data['segments']

    data_list = []
    for segment in segments:
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text']
        confidence = segment['confidence']
        speaker_label = segment['speaker']['label']
        text_edited = segment['textEdited']
        data_list.append([start_time, end_time, text, confidence, speaker_label, text_edited])

    df = pd.DataFrame(data_list, columns=['시작 시간', '종료 시간', '텍스트', '신뢰도', '화자', '텍스트 편집 여부'])
    transcript_file = base_filename + 'speech.xlsx'
    df.to_excel(transcript_file, index=False)

    text_edited_column = df['텍스트 편집 여부']
    transcript_text = base_filename + 'speech.txt'
    with open(transcript_text, 'w', encoding='utf-8') as file:
        for item in text_edited_column:
            file.write(str(item) + '\n')


In [ ]:
if __name__ == '__main__':
    res = ClovaSpeechClient().req_upload(file=audio_file, completion='sync')


# 네이버 클로바 음성 인식 API의 출력 결과
clova_output = res.text

# JSON 형식으로 파싱
data = json.loads(clova_output)

# 필요한 데이터 추출
segments = data['segments']

# 데이터를 저장할 리스트 초기화
data_list = []

# 각 세그먼트의 데이터 추출
for segment in segments:
    start_time = segment['start']
    end_time = segment['end']
    text = segment['text']
    confidence = segment['confidence']
    speaker_label = segment['speaker']['label']
    text_edited = segment['textEdited']

    # 추출한 데이터를 리스트에 추가
    data_list.append([start_time, end_time, text, confidence, speaker_label, text_edited])

# 데이터프레임 생성
df = pd.DataFrame(data_list, columns=['시작 시간', '종료 시간', '텍스트', '신뢰도', '화자', '텍스트 편집 여부'])

transcript_file = base_filename + 'speech.xlsx'

# Excel 파일로 저장
df.to_excel(transcript_file, index=False)


In [ ]:
# '텍스트 편집 여부' 컬럼 추출
text_edited_column = df['텍스트 편집 여부']
transcript_text = base_filename + 'speech.txt'

# 텍스트 파일로 저장
with open(transcript_text, 'w', encoding='utf-8') as file:
    for item in text_edited_column:
        file.write(str(item) + '\n')


# 4.Quantative Analysis
- Tokenizing - count by type
- Drawing wordcloud

- 형태소 분석

In [ ]:
import os
import pandas as pd
from zipfile import ZipFile

# 특정 폴더 내의 모든 txt 파일의 경로를 가져오는 함수
def get_txt_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]

# 텍스트 파일을 읽는 함수
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# analyze_text_with_mecab_to_xlsx 함수 사용

# txt 파일을 순차적으로 읽어 형태소 분석 후 xlsx로 저장하고 zip 파일로 압축하는 함수
def process_files(folder_path):
    txt_files = get_txt_files(folder_path)
    xlsx_files = []

    # 각 txt 파일에 대해 처리
    for txt_file in txt_files:
        text = read_text_file(txt_file)
        # 파일명 추출 (확장자 제외)
        base_name = os.path.splitext(os.path.basename(txt_file))[0]
        output_xlsx = os.path.join(folder_path, base_name + '.xlsx')
        analyze_text_with_mecab_to_xlsx(text, output_xlsx)
        xlsx_files.append(output_xlsx)

    # 모든 xlsx 파일을 하나의 zip 파일로 압축
    zip_filename = os.path.join(folder_path, 'mecab_analysis_results.zip')
    with ZipFile(zip_filename, 'w') as zipf:
        for file in xlsx_files:
            zipf.write(file, os.path.basename(file))
            os.remove(file)  # Optionally remove the file after adding to the zip

    return zip_filename

# 실행 부분
if __name__ == '__main__':
    folder_path = '/content/drive/Shareddrives/커넥츠랩/서비스 테스트(2312)/Recordings/d10/d10_Quan'  # 텍스트 파일이 있는 폴더 경로로 변경
    zip_file = process_files(folder_path)
    print(f"Created {zip_file}")


In [ ]:
# txt 파일 경로 지정
text_file = '/content/drive/Shareddrives/커넥츠랩/서비스 테스트(2312)/Recordings/Report_Final/정석준_speech_Final_Report.txt'  # 분석할 텍스트 파일 경로



In [ ]:
# Extracting the specified part
data_info = extract_last_part(text_file)

base_filename = f"{data_info}_"

text = read_text_file(text_file)
#output_xlsx = base_filename + 'QuanA.xlsx'  # 저장할 XLSX 파일 경로
#analyze_text_with_mecab_to_xlsx(text, output_xlsx)

# Drawing WordCloud
output_png = base_filename + 'wc.png'  # 저장할 XLSX 파일 경로

with open(text_file, 'r', encoding='utf-8') as f:
    text = f.read()

okt = Okt()
nouns = okt.nouns(text) # 명사만 추출

words = [n for n in nouns if len(n) > 1]

c = Counter(words)

c_n = Counter(nouns)
c_n_most = dict(c_n.most_common(n=10))
c_n_most

c_most = dict(c.most_common(n=10))
c_most

c_20 = pd.DataFrame([c_most])

wc = WordCloud(font_path='/content/drive/MyDrive/01 커넥츠랩/30 서비스 개발/01 TalkD/21 Code/01_Module_QuantitiveAnalysis/WordCloud/AppleSDGothicNeo.ttc', background_color='white', width=600, height=600, scale=1.0, max_font_size=200, colormap='summer')
gen = wc.generate_from_frequencies(c)
plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(gen)

wc.generate_from_frequencies(c)

output_png = base_filename + 'wc.png'  # 저장할 wordcloud 파일 경로

wc.to_file(output_png)  # 워드클라우드 생성 및 저장
